In [3]:
pip install numpy==1.21.6
pip install opencv-python

In [98]:
from osgeo_utils.gdal_pansharpen import gdal_pansharpen
import rasterio
from rasterio.merge import merge
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform
from osgeo import gdal
import rioxarray
import xarray as xr
from Simple_Pansharpen import *

In [3]:
%load_ext lab_black

Here I am creating a wrapper function to include both pansharpen() and gdal_pansharpen(). The pansharpen() function was taken from Thomas Wang's Simple_Pansharpen.py script (https://github.com/ThomasWangWeiHong/Simple-Pansharpening-Algorithms/blob/master/Simple_Pansharpen.py). For the purpose of this project, only the 'Simple Mean' method was considered.

This block of code is working for now only for the simple mean. As gdal_pansharpen() receives a list (sequence) as a parameter for spectral_names and pansharpen() a string, I have not found a way to reconcile both. I tried to put 'spectral_names' inside of a list as shown in the code, but it isn't working. I also tried to call the function as the following example and again it produced a 'Not a sequence' error message.

wrapper_pansharpen(
    pan_name="panchr.tif",
    spectral_names=["multispectral.tif"],
    dst_filename="psh.tif",
    simple_mean=False,
)

In [92]:
# If user selects Simple Mean pansharpening, Thomas Wang's method is applied. Otherwise, gdal_pansharpen() is used
def wrapper_pansharpen(
    pan_name,
    dst_filename,
    spectral_names,
    simple_mean=True,
    band_nums=None,
    weights=None,
    resampling=None,
    spat_adjust=None,
    bitdepth=None,
    nodata_value=False,
):
    """
    Inputs:

    This function combines the simple_mean pansharpening developed by Thomas Wang and the pansharpening tools from GDAL.

    - pan_name: File path of the higher resolution image to be used for pansharpening
    - spectral_names: File path of the coarser image to undergo pansharpening
    - band_nums: bands in the coarser image to undergo pansharpening when not applied to the whole dataset
    - weights: Specify a weight for the computation of the pseudo panchromatic value. There must be as many -w switches as input spectral bands
    - dst_filename: File path of pansharpened dataset to be written to file
    - resampling: Select a resampling algorithm (nearest, bilinear, cubic [default], cubicspline, lanczos, average)
    - spat_adjust: Select behavior when bands have not the same extent (union [default], intersection, none, nonewithoutwarning)
    - bitdepth: Specify the bit depth of the panchromatic and spectral bands (e.g. 12). If not specified, the NBITS metadata item from the panchromatic band will be used if it exists.
    - nodata_value: Specify nodata value for bands. Used for the resampling and pan-sharpening computation itself. If not set, deduced from the input bands, provided they have a consistent setting.
    - simple_mean: if True, pansharpening is performed using the pansharpen_simple_mean() function. Otherwise, gdal_pansharpen() is selected

    """
    if simple_mean == True:
        pansharpen(spectral_names, pan_name, dst_filename, method="simple_mean")
    else:
        gdal_pansharpen(
            pan_name,
            [spectral_names],
            band_nums,
            weights,
            dst_filename,
            resampling,
            spat_adjust,
            bitdepth,
            nodata_value,
        )

    return dst_filename

This function is writing the coordinate system but the rasters are displayed on the equator in ArcGIS. I know what the problem is! The function below is not writing the extent from the panchromatic band to the output. To solve this, I tried to use rasterio.transform (https://rasterio.readthedocs.io/en/latest/api/rasterio.transform.html#rasterio.transform.AffineTransformer) but I got an "'Affine' object has no attribute 'AffineTransformer'" error message. I also tried to use rasterio.merge.merge but it also yielded errors and the documentation says 'All files must have the same number of bands', which is not this case as we are trying to stack a multiband raster with a monoband raster. 

In [63]:
# When the user wants to stack the multispectral with the panchromatic bands into one single raster dataset
def stack_bands(pan_name, spectral_names, dst_filename):
    """
    Inputs:
    - pan_name: File path of the higher resolution image (panchromatic band)
    - spectral_names: File path of the pansharpened image
    - dst_filename: File path of the stacked dataset
    """
    bands = list()
    panchromatic = rioxarray.open_rasterio(pan_name)
    multispectral = rioxarray.open_rasterio(spectral_names)

    # All the bands are added to a list
    bands.append(panchromatic.sel(band=1))

    for file in range(multispectral.rio.count):
        bands.append(multispectral.sel(band=file + 1))

    # The bands are stacked into a single array
    bands_array = xr.DataArray(bands)
    stack = bands_array.stack()

    # Spatial reference
    crs = int(panchromatic.rio.crs.to_proj4()[11:])
    stack.rio.write_crs(crs, inplace=True).rio.set_spatial_dims(
        y_dim=stack.dims[1], x_dim=stack.dims[2]
    ).rio.write_coordinate_system(inplace=True)

    print(stack.rio.bounds)
    print(type(stack))

    # The data array is converted to dataset
    stack.rio.to_raster(dst_filename)

    dstRst = rasterio.open(panchromatic)
    srcRst = rasterio.open(dst_filename)
    dstCrs = dstRst.crs

    transform, width, height = calculate_default_transform(
        srcRst.crs, dstCrs, srcRst.width, srcRst.height, *srcRst.bounds
    )

    srcRst.transform

In [76]:
raster = rasterio.open("panchr.tif")
transform, width, height = calculate_default_transform(
    raster.crs, dstCrs, raster.width, raster.height, *raster.bounds
)
transform

Affine(14.999999999999998, 0.0, 441787.5,
       0.0, -14.999999999999998, 4953922.5)

In [86]:
file = rasterio.open("pan.tif")
file.transform.AffineTransformer(transform)

AttributeError: 'Affine' object has no attribute 'AffineTransformer'

In [82]:
rasterio.transform?

Type:        module
String form: <module 'rasterio.transform' from '/opt/conda/lib/python3.9/site-packages/rasterio/transform.py'>
File:        /opt/conda/lib/python3.9/site-packages/rasterio/transform.py
Docstring:   Geospatial transforms


I also tried this approach to modify the extent of the output, but it created an empty file at first and now it shows a 'panDstBands[0] = 2 ... out of range for dataset.' error.

In [71]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# open source raster
srcRst = rasterio.open("pan.tif")
print("source raster crs:")
print(srcRst.crs)

dstRst = rasterio.open("panchr.tif")
dstCrs = {"init": "EPSG:32615"}
print("destination raster crs:")
print(dstCrs)

# calculate transform array and shape of reprojected raster
transform, width, height = calculate_default_transform(
    dstRst.crs, dstCrs, dstRst.width, dstRst.height, *dstRst.bounds
)
print("transform array of source raster")
print(srcRst.transform)

print("transform array of destination raster")
print(transform)

# working of the meta for the destination raster
kwargs = dstRst.meta.copy()
kwargs.update(
    {
        "crs": dstCrs,
        "transform": transform,
        "width": width,
        "height": height,
    }
)
# open destination raster
output = rasterio.open("output.tif", "w", **kwargs)
# reproject and save raster band data
for i in range(1, srcRst.count + 1):
    reproject(
        source=rasterio.band(srcRst, i),
        destination=rasterio.band(output, i),
        # src_transform=srcRst.transform,
        src_crs=srcRst.crs,
        # dst_transform=transform,
        dst_crs=dstCrs,
        resampling=Resampling.nearest,
    )
# close destination raster
dstRst.close()

source raster crs:
EPSG:32615
destination raster crs:
{'init': 'EPSG:32615'}
transform array of source raster
| 15.00, 0.00, 441787.50|
| 0.00,-15.00, 4953922.50|
| 0.00, 0.00, 1.00|
transform array of destination raster
| 15.00, 0.00, 441787.50|
| 0.00,-15.00, 4953922.50|
| 0.00, 0.00, 1.00|


CPLE_IllegalArgError: panDstBands[0] = 2 ... out of range for dataset.

In [45]:
merge(['panchr.tif', 'pan.tif'])

DatasetIOShapeError: Dataset indexes and destination buffer are mismatched

In [99]:
merge?

Signature:
merge(
    datasets,
    bounds=None,
    res=None,
    nodata=None,
    dtype=None,
    precision=None,
    indexes=None,
    output_count=None,
    resampling=<Resampling.nearest: 0>,
    method='first',
    target_aligned_pixels=False,
    dst_path=None,
    dst_kwds=None,
)
Docstring:
Copy valid pixels from input files to an output file.

All files must have the same number of bands, data type, and
coordinate reference system.

Input files are merged in their listed order using the reverse
painter's algorithm (default) or another method. If the output file exists,
its values will be overwritten by input values.

Geospatial bounds and resolution of a new output file in the
units of the input file coordinate reference system may be provided
and are otherwise taken from the first input file.

Parameters
----------
datasets : list of dataset objects opened in 'r' mode, filenames or pathlib.Path objects
    source datasets to be merged.
bounds: tuple, optional
    Bounds of t

In [6]:
# List of resampling algorithms included in rasterio
resampling_techniques = [
    Resampling.nearest,
    Resampling.bilinear,
    Resampling.cubic,
    Resampling.cubic_spline,
    Resampling.lanczos,
    Resampling.average,
    Resampling.mode,
    Resampling.gauss,
    Resampling.max,
    Resampling.min,
    Resampling.med,
    Resampling.q1,
    Resampling.q3,
    Resampling.sum,
    Resampling.rms,
]

In [7]:
# Generates raster overviews
def create_pyramids(ds, resampling):
    """
    Inputs:
    - ds: File path of pansharpened dataset
    - resampling: Select a resampling algorithm
    """
    factors = [2, 4, 8, 16]
    for i in resampling_techniques:
        if resampling == str(resampling_techniques[i])[11:]:
            dst = rasterio.open(ds, "r+")
            dst.build_overviews(factors, resampling_techniques[i])
            dst.update_tags(
                ns="rio_overview", resampling=str(resampling_techniques[i])[11:]
            )
            dst.close()
            break  # exit out of loop here

## Resampling
In addition to the pansharpening tool, this project aims to create an up-and-downsampling tool where the users can change (resample) the cell size of their raster datasets without external bands.

The next cell was based on https://rasterio.readthedocs.io/en/latest/topics/resampling.html and https://pygis.io/docs/e_raster_resample.html. However, the function was unable to write the output raster successfully.

In [8]:
def resample(factor, ds, resampling):
    """
    Inputs:
    - factor: numeric entry to upsample/downsample the raster
    - ds: File path of raster dataset
    - resampling: Select a resampling algorithm

    """
    upscale_factor = factor

    for i in resampling_techniques:
        if r_technique == str(resampling_techniques[i])[11:]:

            with rasterio.open(ds) as dataset:

                # resample data to target shape
                data = dataset.read(
                    out_shape=(
                        dataset.count,
                        int(dataset.height * upscale_factor),
                        int(dataset.width * upscale_factor),
                    ),
                    resampling=resampling_techniques[i],
                )

                # scale image transform
                transform = dataset.transform * dataset.transform.scale(
                    (dataset.width / data.shape[-1]), (dataset.height / data.shape[-2])
                )

                ## Write outputs
                # set properties for output
                dst_kwargs = dataset.meta.copy()

                ### the error source is here
                dst_kwargs.update(
                    {
                        "crs": dataset.crs,
                        "transform": transform,
                        "width": data.shape[-1],
                        "height": data.shape[-2],
                        "nodata": 0,
                    }
                )

                with rasterio.open("resampled_raster.tif", "w", **dst_kwargs) as dst:
                    # iterate through bands
                    for i in range(data.shape[0]):
                        dst.write(data[i].astype(rasterio.uint32), i + 1)

                break

A new tool, gdal.Translate(), was found to create the resampled raster successfully. This tool was i) simplified to receive just a few parameters and ii) complemented with the create_pyramids() function. 

For now, this function is writing an output raster with the cells resized, but it's creating the stats for the input raster instead of the output, and ArcMap shows the pyramids are absent.

In [9]:
def resize(
    output_name,
    ds,
    Res,
    resampling,
    of,
    ot=None,
    statistics=False,
    pyramids=False,
    resampling_pyramids=None,
):
    """
    Inputs:
    - output_name: File path to the output raster resized
    - ds: File path of input dataset
    - Res: Set the size of the cells
    - resampling: Select a resampling algorithm (nearest [default], bilinear, cubic, cubicspline, lanczos, average, rms, mode)
    - of: output format
    - ot: output type (Byte, UInt16, Int16, UInt32, Int32, Float32, Float64, CInt16, CInt32, CFloat32 or CFloat64)
    - statistics: whether to calculate statistics
    - pyramids: whether to calculate pyramids
    - resampling_pyramids: resampling algorithm if pyramids == True

    """

    gdal.Translate(
        output_name,
        ds,
        xRes=Res,
        yRes=Res,
        resampleAlg=resampling,
        format=of,
        stats=statistics,
    )

    if pyramids == True:
        create_pyramids(output_name, resampling_pyramids)

In [43]:
tifs = ['panchr.tif', 'pan.tif']
OUTS = 
gdal.Translate(tifs, output_name='stack.tif', format = 'GTiff')

TypeError: Translate() missing 1 required positional argument: 'srcDS'

In [16]:
gdal.TranslateOptions?

Signature:
gdal.TranslateOptions(
    options=None,
    format=None,
    outputType=0,
    bandList=None,
    maskBand=None,
    width=0,
    height=0,
    widthPct=0.0,
    heightPct=0.0,
    xRes=0.0,
    yRes=0.0,
    creationOptions=None,
    srcWin=None,
    projWin=None,
    projWinSRS=None,
    strict=False,
    unscale=False,
    scaleParams=None,
    exponents=None,
    outputBounds=None,
    metadataOptions=None,
    outputSRS=None,
    nogcp=False,
    GCPs=None,
    noData=None,
    rgbExpand=None,
    stats=False,
    rat=True,
    resampleAlg=None,
    callback=None,
    callback_data=None,
)
Docstring:
Create a TranslateOptions() object that can be passed to gdal.Translate()
Keyword arguments are :
  options --- can be be an array of strings, a string or let empty and filled from other keywords.
  format --- output format ("GTiff", etc...)
  outputType --- output type (gdalconst.GDT_Byte, etc...)
  bandList --- array of band numbers (index start at 1)
  maskBand --- mask